##구글 드라이브 마운트

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [5]:
FACE_width=46
FACE_height=56
FACE_size=(FACE_width,FACE_height)
FACE_channels=3

In [6]:
IRIS_width=192
IRIS_height=144
IRIS_size=(IRIS_width,IRIS_height)
IRIS_channels=3

In [7]:
IMAGE_width=100
IMAGE_height=100
IMAGE_size=(IMAGE_width,IMAGE_height)
IMAGE_channels=3

## 파일 경로 지정

In [8]:
train_path='/gdrive/My Drive/생체인증보안/멀티모달/train_set'
test_path='/gdrive/My Drive/생체인증보안/멀티모달/test_set'

## 라이브러리 import

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
from keras.preprocessing import image
from keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import tensorflow as tf
import random
import glob, os
import cv2
from PIL import Image

## Augmentation 후 저장

In [ ]:
# 랜덤시드 고정시키기
np.random.seed(50)

from keras.preprocessing.image import ImageDataGenerator, img_to_array

# 데이터셋 불러오기
data_aug_gen = ImageDataGenerator(brightness_range=[0.8,1.2],
                                  zoom_range=[0.9, 1.1],
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  fill_mode='constant')
 

In [ ]:
%cd '/gdrive/My Drive/생체인증보안/멀티모달/train_set'
nowpath='/gdrive/My Drive/생체인증보안/멀티모달/train_set'

for filename in os.listdir(nowpath):
  dtype=filename[4:8]
  if dtype=="iris":
    img=Image.open(filename)
    img = img_to_array(img)
    img = img.reshape((1,) + img.shape)
    i = 0
  # 이 for는 무한으로 반복되기 때문에 우리가 원하는 반복횟수를 지정하여, 지정된 반복횟수가 되면 빠져나오도록 해야합니다.
    for batch in data_aug_gen.flow(img, batch_size=1, save_to_dir='./', save_prefix=filename, save_format='png'):
      i += 1
      if i >4: 
           break

/gdrive/My Drive/생체인증보안/멀티모달/train_set


In [ ]:
images=sorted(glob.glob("*"))

print(len(images))

3072


##데이터 불러오기

In [9]:
import natsort
f_x_data=[]
f_y_data=[]

i_x_data=[]
i_y_data=[]

fname=[]
for filename in os.listdir(train_path):
  fname.append(filename)

fname=natsort.natsorted(fname)

for file_name in fname:
    file_path='{}/{}'.format(train_path, file_name)
    image=Image.open(file_path).convert("RGB")

    name=file_name[0:3]
    y_name=int(name)
    dtype=file_name[4:8]

    if dtype=="face":
      image=image.resize((IMAGE_width,IMAGE_height))
      np_image=np.asarray(image)
      np_image=np_image/255
      f_x_data.append(np_image)
      f_y_data.append(y_name)
    elif dtype=="iris":
      image=image.resize((IMAGE_width,IMAGE_height))
      np_image=np.asarray(image)
      np_image=np_image/255
      i_x_data.append(np_image)
      i_y_data.append(y_name)
  

In [10]:
f_x_data = np.array(f_x_data)
f_y_data = np.array(f_y_data)

i_x_data = np.array(i_x_data)
i_y_data = np.array(i_y_data)

In [11]:
f_x_data.shape

(1536, 100, 100, 3)

In [12]:
i_x_data.shape

(1536, 100, 100, 3)

In [13]:
x_data=np.concatenate([f_x_data,i_x_data],axis=2)
y_data=f_y_data

In [14]:
x_data.shape

(1536, 100, 200, 3)

In [15]:
y_data.shape

(1536,)

In [ ]:
print(f_y_data)

[ 0  0  0 ... 63 63 63]


In [ ]:
print(i_y_data)

[ 0  0  0 ... 63 63 63]


In [16]:
np.save("/gdrive/My Drive/생체인증보안/멀티모달/x_data.npy",x_data)
np.save("/gdrive/My Drive/생체인증보안/멀티모달/y_data.npy",y_data)
np.save("/gdrive/My Drive/생체인증보안/멀티모달/f_x_data-1.npy",f_x_data)
np.save("/gdrive/My Drive/생체인증보안/멀티모달/f_y_data-1.npy",f_y_data)
np.save("/gdrive/My Drive/생체인증보안/멀티모달/i_x_data-1.npy",i_x_data)
np.save("/gdrive/My Drive/생체인증보안/멀티모달/i_y_data-1.npy",i_y_data)

In [ ]:
x_data = np.load('/gdrive/My Drive/생체인증보안/멀티모달/x_data.npy')
y_data = np.load('/gdrive/My Drive/생체인증보안/멀티모달/y_data.npy')
f_x_data = np.load('/gdrive/My Drive/생체인증보안/멀티모달/f_x_data-1.npy')
f_y_data = np.load('/gdrive/My Drive/생체인증보안/멀티모달/f_y_data-1.npy')
i_x_data = np.load('/gdrive/My Drive/생체인증보안/멀티모달/i_x_data-1.npy')
i_y_data = np.load('/gdrive/My Drive/생체인증보안/멀티모달/i_y_data-1.npy')

##모델링

In [17]:
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet169, DenseNet201, DenseNet121
from tensorflow.keras.layers import Conv2D, Input, Flatten, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import losses, optimizers, activations, metrics

In [18]:
#input 정의
input=Input(shape=(IMAGE_height,IMAGE_width*2,IMAGE_channels))

base_model=DenseNet121(input_shape=(IMAGE_height,IMAGE_width*2,IMAGE_channels),
                       input_tensor=input,include_top=False)

x = base_model.output
out1 = GlobalMaxPooling2D()(x)
out2 = GlobalAveragePooling2D()(x)
out3 = Flatten()(x)
out = Concatenate(axis=-1)([out1, out2, out3])
out = Dropout(0.5)(out)

predictions = Dense(64, activation='softmax',kernel_regularizer='l2')(out)
model = Model(inputs=base_model.input, outputs=predictions)

adam=optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.summary()

29089792/29084464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 200, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 106, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 50, 100, 64)  9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 50, 100, 64)  256         conv1/conv[0][0]                 
_____________________________

##Callback

In [19]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
#조기종료
earlystop=EarlyStopping(monitor='sparse_categorical_accuracy',patience=10)

#학습율 조정
learning_rate_reduction=ReduceLROnPlateau(monitor='sparse_categorical_accuracy',
                                          patience=2,
                                          verbose=1,
                                          factor=0.5,
                                          min_lr=0.000001)
#callback 설정
callback=[earlystop,learning_rate_reduction]

##모델 학습(Cross-Validation)

In [20]:
from sklearn.model_selection import StratifiedKFold

kfold=StratifiedKFold(n_splits=3)
cvscores = []

for train_idx,val_idx in kfold.split(x_data,y_data):
  x_train,x_val=x_data[train_idx],x_data[val_idx]
  y_train,y_val=y_data[train_idx],y_data[val_idx]

  model.fit(x_train,y_train,epochs=6,batch_size=16,verbose=1,callbacks=callback)
  
  scores = model.evaluate(x_val, y_val, verbose=1)
  print("Acc: ",(scores[1]*100))
  cvscores.append(scores[1] * 100)

Epoch 1/6
64/64 [==============================] - 48s 92ms/step - loss: 3.6670 - sparse_categorical_accuracy: 0.4971
Epoch 2/6
64/64 [==============================] - 6s 93ms/step - loss: 0.9131 - sparse_categorical_accuracy: 0.9854
Epoch 3/6
64/64 [==============================] - 6s 93ms/step - loss: 0.7607 - sparse_categorical_accuracy: 0.9932
Epoch 4/6
64/64 [==============================] - 6s 94ms/step - loss: 0.7021 - sparse_categorical_accuracy: 0.9971
Epoch 5/6
64/64 [==============================] - 6s 94ms/step - loss: 0.6663 - sparse_categorical_accuracy: 0.9990
Epoch 6/6
16/16 [==============================] - 3s 44ms/step - loss: 0.6426 - sparse_categorical_accuracy: 1.0000
Acc:  100.0
Epoch 1/6
64/64 [==============================] - 6s 94ms/step - loss: 0.6382 - sparse_categorical_accuracy: 0.9961
Epoch 2/6
64/64 [==============================] - 6s 94ms/step - loss: 0.6063 - sparse_categorical_accuracy: 0.9990
Epoch 3/6
64/64 [==============================] - 

In [21]:
print("평균 검증 Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

평균 검증 Accuracy: 100.00% (+/- 0.00%)


In [ ]:
%cd '/gdrive/My Drive/생체인증보안/멀티모달/'
model.save('multimodal_rec.h5')

/gdrive/My Drive/생체인증보안/멀티모달


##모델 성능 지표</br>
학습 데이터 중 split한 validation 데이터 사용

In [23]:
y_train_pred=model.predict(x_train)
y_train_pred

array([[9.9999368e-01, 5.0616122e-10, 3.6238664e-08, ..., 4.2895248e-08,
        4.2181170e-09, 3.9312673e-09],
       [9.9999726e-01, 7.7832141e-10, 4.1996122e-09, ..., 1.5949029e-08,
        5.7289342e-09, 4.1469024e-09],
       [9.9995315e-01, 1.5511461e-08, 1.7141851e-07, ..., 4.9145700e-07,
        1.4741109e-07, 8.6813856e-08],
       ...,
       [2.1124627e-08, 2.3649347e-08, 1.2564999e-06, ..., 9.7307009e-08,
        7.9329698e-09, 9.9998331e-01],
       [6.6343908e-10, 1.0580958e-09, 3.9137380e-08, ..., 2.0485167e-08,
        1.8141706e-10, 9.9999893e-01],
       [3.3239533e-11, 3.9313636e-11, 5.8151954e-09, ..., 2.9254139e-09,
        7.6427233e-12, 1.0000000e+00]], dtype=float32)

In [24]:
print(y_train)

[ 0  0  0 ... 63 63 63]


In [25]:
y_pred_train=[]
for i in range(len(y_train)):
  n=np.argmax(y_train_pred[i])
  y_pred_train.append(n)

In [26]:
print(y_pred_train)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18,

In [28]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_train,y_pred_train)                         
precision = precision_score(y_train,y_pred_train,average='micro') 
recall = recall_score(y_train,y_pred_train, average='micro')      
f1 = f1_score(y_train,y_pred_train, average='micro')

print('정확도: {0:.6f}, 정밀도: {1:.6f}, 재현율: {2:.6f}, F1-score: {3:.6f}'.
      format(accuracy, precision, recall, f1))

정확도: 1.000000, 정밀도: 1.000000, 재현율: 1.000000, F1-score: 1.000000


##테스트

In [30]:
import natsort
%cd '/gdrive/My Drive/생체인증보안/멀티모달/test_set'

fname=[]
f_name=[]
i_name=[]

f_x_test=[]
f_y_test=[]
i_x_test=[]
i_y_test=[]

for filename in os.listdir(test_path):
  fname.append(filename)

fname=natsort.natsorted(fname)

for filename in fname:
  img=Image.open(filename).convert("RGB")
  print(filename)
  if 'face' in filename:
    img=img.resize((IMAGE_width,IMAGE_height))
    np_image=np.asarray(img)
    np_image=np_image/255
    f_x_test.append(np_image)
    f_name.append(filename)
  elif 'iris' in filename:
    img=img.resize((IMAGE_width,IMAGE_height))
    np_image=np.asarray(img)
    np_image=np_image/255
    i_x_test.append(np_image)
    i_name.append(filename)
  
f_x_test = np.array(f_x_test)
i_x_test = np.array(i_x_test)

/gdrive/My Drive/생체인증보안/멀티모달/test_set
0_face.png
0_iris.png
1_face.png
1_iris.png
2_face.png
2_iris.png
3_face.png
3_iris.png
4_face.png
4_iris.png
5_face.png
5_iris.png
6_face.png
6_iris.png
7_face.png
7_iris.png
8_face.png
8_iris.png
9_face.png
9_iris.png
10_face.png
10_iris.png
11_face.png
11_iris.png
12_face.png
12_iris.png
13_face.png
13_iris.png
14_face.png
14_iris.png
15_face.png
15_iris.png
16_face.png
16_iris.png
17_face.png
17_iris.png
18_face.png
18_iris.png
19_face.png
19_iris.png
20_face.png
20_iris.png
21_face.png
21_iris.png
22_face.png
22_iris.png
23_face.png
23_iris.png
24_face.png
24_iris.png
25_face.png
25_iris.png
26_face.png
26_iris.png
27_face.png
27_iris.png
28_face.png
28_iris.png
29_face.png
29_iris.png
30_face.png
30_iris.png
31_face.png
31_iris.png
32_face.png
32_iris.png
33_face.png
33_iris.png
34_face.png
34_iris.png
35_face.png
35_iris.png
36_face.png
36_iris.png
37_face.png
37_iris.png
38_face.png
38_iris.png
39_face.png
39_iris.png
40_face.png
40_iris.pn

In [31]:
f_x_test.shape

(64, 100, 100, 3)

In [32]:
i_x_test.shape

(64, 100, 100, 3)

In [33]:
x_test=np.concatenate([f_x_test,i_x_test],axis=2)
y_test=f_y_test

In [34]:
x_test.shape

(64, 100, 200, 3)

In [35]:
y_pred=model.predict(x_test)
print(y_pred)

[[4.6501225e-07 1.0353799e-06 4.9358246e-06 ... 1.5694501e-05
  3.6590055e-07 4.4571789e-07]
 [6.2370957e-05 3.5139228e-06 1.1463385e-06 ... 1.2098640e-06
  1.2445945e-07 4.2274519e-07]
 [1.4316407e-07 3.7873920e-07 4.7228063e-08 ... 2.1158722e-07
  3.2418180e-07 2.4893865e-07]
 ...
 [6.2746717e-06 5.2852042e-06 7.5121202e-06 ... 2.8272590e-04
  1.1365046e-06 9.2058763e-06]
 [3.7685320e-06 1.0329649e-04 1.5009932e-05 ... 5.8724945e-06
  2.2307590e-06 2.8213508e-06]
 [1.2601495e-06 3.3884072e-07 8.7093758e-07 ... 2.6566150e-07
  2.9073228e-06 1.6632600e-07]]


In [36]:
result=[]
for i in range(64):
  n=np.argmax(y_pred[i])
  result.append(n)

In [37]:
print(result)

[27, 31, 22, 57, 54, 45, 5, 12, 8, 7, 36, 2, 46, 50, 10, 61, 49, 59, 56, 13, 37, 30, 25, 0, 41, 58, 9, 20, 48, 47, 42, 24, 3, 6, 55, 63, 21, 16, 34, 11, 35, 32, 40, 62, 19, 52, 33, 15, 1, 29, 17, 26, 53, 28, 18, 14, 43, 60, 4, 39, 38, 23, 51, 44]


In [38]:
list_df = pd.DataFrame({'Answer':result})
list_df.head(20)

,Answer
0,27
1,31
2,22
3,57
4,54
5,45
6,5
7,12
8,8
9,7


In [ ]:
list_df.tail(20)

,Answer
44,19
45,52
46,33
47,15
48,1
49,29
50,17
51,26
52,53
53,28


In [39]:
list_df.to_csv("/gdrive/My Drive/생체인증보안/멀티모달/result2-모델1개.csv")